[【官方双语】深度学习之反向传播算法 上/下](https://b23.tv/2lQ1RK)
# autograd——自动求导系统

In [1]:
import torch
torch.manual_seed(7)

1. torch.autograd.backward(tensors, grad_tensors=None, retain_graph=None, create_graph=False)
    - 功能：自动求取梯度
        - tensors：用于求导的张量，如 loss
        - retain_graph：保存计算图
        - create_graph：创建导数计算图，用于高阶求导
        - grad_tensors：多梯度权重

In [2]:
w = torch.tensor([1.], requires_grad=True)
x = torch.tensor([2.], requires_grad=True)

a = torch.add(w, x)
b = torch.add(w, 1)
y = torch.mul(a, b)

y.backward(retain_graph=True)

2. torch.autograd.grad(outputs, inputs, grad_outputs=None, retain_graph=None, create_graph=False)
    - 功能：求取梯度
        - outputs：用于求导的张量，如 loss
        - inputs：需要梯度的张量
        - create_graph：创建导数计算图，用于高阶求导
        - retain_graph：保存计算图
        - grad_outputs：多梯度权重

In [3]:
x = torch.tensor([3.], requires_grad=True)
y = torch.pow(x, 2)     # y = x**2

grad_1 = torch.autograd.grad(y, x, create_graph=True)   # grad_1 = dy/dx = 2x = 2 * 3 = 6
grad_2 = torch.autograd.grad(grad_1[0], x)              # grad_2 = d(dy/dx)/dx = d(2x)/dx = 2

autograd小贴士：
1. 梯度不自动清零
2. 依赖于叶子节点的节点，requires_grad默认为True
3. 叶子节点不可执行in-place

- autograd 包是 PyTorch 中所有神经网络的核心。
- 首先让我们简要地介绍它，然后我们将会去训练我们的第一个神经网络。该 autograd 软件包为 Tensors 上的所有操作提供自动微分。它是一个由运行定义的框架，这意味着以代码运行方式定义你的后向传播，并且每次迭代都可以不同。
- 我们从 tensor 和 gradients 来举一些例子。
    - torch.Tensor是包的核心类。如果将其属性.requires_grad设置为True，则会开始跟踪针对tensor的所有操作。完成计算后，你可以调用.backward()来自动计算所有梯度。该张量的梯度将累积到.grad属性中。
    - 要停止tensor历史记录的跟踪，你可以调用.detach()，它将其与计算历史记录分离，并防止将来的计算被跟踪。
    - 要停止跟踪历史记录（和使用内存），你还可以将代码块使用with torch.no_grad()：包装起来。在评估模型时，这是特别有用，因为模型在训练阶段具有requires_grad=True的可训练参数有利于调参，但在评估阶段我们不需要梯度。
    - 还有一个类对于autograd实现非常重要那就是Function。Tensor和Function互相连接并构建一个非循环图，它保存整个完整的计算过程的历史信息。每个张量都有一个.grad_fn属性保存着创建了张量的Function的引用（如果用户自己创建张量，则grad_fn是None）。
    - 如果你想计算导数，你可以调用Tensor.backward()。如果Tensor是标量（即它包含一个元素数据），则不需要指定任何参数backward()，但是如果它有更多元素，则需要指定一个gradient参数来指定张量的形状。
- 现在让我们看一个雅可比向量积的例子：

In [4]:
x = torch.randn(3, requires_grad=True)

y = x * 2
while y.data.norm() < 1000:
    y = y * 2

print(y)

tensor([-150.3182,  805.0087,  969.5453], grad_fn=<MulBackward0>)


- 现在在这种情况下，y不再是一个标量。torch.autograd不能够直接计算整个雅可比，但是如果我们只想要雅可比向量积，只需要简单地传递向量给backward作为参数。

In [5]:
v = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
y.backward(v)

print(x.grad)

tensor([1.0240e+02, 1.0240e+03, 1.0240e-01])


- 你可以通过将代码包裹在with torch.no_grad()，来停止对从跟踪历史中的.requires_grad=True的张量自动求导。

In [8]:
print(x.requires_grad)
print((x ** 2).requires_grad)

with torch.no_grad():
    print((x ** 2).requires_grad)

True
True
False


# 练习题
求$y=x^2$对$x$的一阶偏导和二阶偏导。

In [9]:
x = torch.tensor([3.], requires_grad=True)
y = torch.pow(x, 2)  # y = x**2

grad_1 = torch.autograd.grad(y, x, create_graph=True)  # grad_1 = dy/dx = 2x = 2 * 3 = 6
grad_2 = torch.autograd.grad(grad_1[0], x)             # grad_2 = d(dy/dx)/dx = d(2x)/dx = 2